In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs
from reversible2.invert import invert

from reversible2.affine import AdditiveBlock


In [ ]:
from reversible2.bhno import create_inputs, load_file

In [ ]:
orig_train_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat')
train_cnt = orig_train_cnt.reorder_channels(['C3', 'C4'])

train_inputs = create_inputs(train_cnt, final_hz=64, half_before=True)

In [ ]:
orig_test_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/test/4.mat')
test_cnt = orig_test_cnt.reorder_channels(['C3', 'C4'])
test_inputs = create_inputs(test_cnt, final_hz=64, half_before=True)

In [ ]:
cuda = False
if cuda:
    train_inputs = [i.cuda() for i in train_inputs]
    test_inputs = [i.cuda() for i in test_inputs]

In [ ]:
train_inputs[0].shape

In [ ]:
bps_train = [th.mean(th.mean(th.abs(th.rfft(ins.squeeze(),signal_ndim=1)), dim=-1)[:,:,9:14], dim=-1)
 for ins in train_inputs]

bps_test = [th.mean(th.mean(th.abs(th.rfft(ins.squeeze(),signal_ndim=1)), dim=-1)[:,:,9:14], dim=-1)
 for ins in test_inputs]

plt.figure(figsize=(7,7))
plt.scatter(var_to_np(bps_train[0])[:,0], var_to_np(bps_train[0])[:,1])
plt.scatter(var_to_np(bps_train[1])[:,0], var_to_np(bps_train[1])[:,1])


plt.scatter(var_to_np(bps_test[0])[:,0], var_to_np(bps_test[0])[:,1])
plt.scatter(var_to_np(bps_test[1])[:,0], var_to_np(bps_test[1])[:,1])

In [ ]:
train_inputs = bps_train
test_inputs = bps_test

In [ ]:
from reversible2.distribution import TwoClassDist

from reversible2.blocks import dense_add_block, conv_add_block_3x3
from reversible2.util import set_random_seeds
import torch as th
cuda = False
set_random_seeds(2019011641, cuda)
feature_model = nn.Sequential(
    dense_add_block(2,64),
    dense_add_block(2,64),
    dense_add_block(2,64),
    dense_add_block(2,64),
)

feature_model.eval()
# Check that forward + inverse is really identical
t_out = feature_model(train_inputs[0][:2])
inverted = invert(feature_model, t_out)
assert th.allclose(train_inputs[0][:2], inverted, rtol=1e-3,atol=1e-4)
device = list(feature_model.parameters())[0].device
from reversible2.ot_exact import ot_euclidean_loss_for_samples
class_dist = TwoClassDist(2, np.prod(train_inputs[0].size()[1:]) - 2)
if cuda:
    class_dist.cuda()
optim_model = th.optim.Adam(feature_model.parameters())
optim_dist = th.optim.Adam(class_dist.parameters(), lr=1e-2)

In [ ]:
from reversible2.graph import Node
from reversible2.branching import CatChans, ChunkChans, Select

def invert(feature_model, samples):
    from reversible2.invert import invert
    return invert(feature_model, samples)

In [ ]:
%%writefile plot.py
import torch as th
import matplotlib.pyplot as plt
import numpy as np
from reversible2.util import var_to_np
from reversible2.plot import display_close
from matplotlib.patches import Ellipse
import seaborn

def plot_outs(feature_model, train_inputs, test_inputs, class_dist):
    
    # Compute dist for mean/std of encodings
    data_cls_dists = []
    for i_class in range(len(train_inputs)):
        this_class_outs = feature_model(train_inputs[i_class])[:,:2]
        data_cls_dists.append(
            th.distributions.MultivariateNormal(th.mean(this_class_outs, dim=0),
            covariance_matrix=th.diag(th.std(this_class_outs, dim=0) ** 2)))
    for setname, set_inputs in (("Train", train_inputs), ("Test", test_inputs)):

        outs = [feature_model(ins) for ins in set_inputs]
        c_outs = [o[:,:2] for o in outs]

        c_outs_all = th.cat(c_outs)

        cls_dists = []
        for i_class in range(len(c_outs)):
            mean, std = class_dist.get_mean_std(i_class)
            cls_dists.append(
                th.distributions.MultivariateNormal(mean[:2],covariance_matrix=th.diag(std[:2] ** 2)))

        preds_per_class = [th.stack([cls_dists[i_cls].log_prob(c_out)
                         for i_cls in range(len(cls_dists))],
                        dim=-1) for c_out in c_outs]

        pred_labels_per_class = [np.argmax(var_to_np(preds), axis=1)
                       for preds in preds_per_class]

        labels = np.concatenate([np.ones(len(set_inputs[i_cls])) * i_cls 
         for i_cls in range(len(train_inputs))])

        acc = np.mean(labels == np.concatenate(pred_labels_per_class))

        data_preds_per_class = [th.stack([data_cls_dists[i_cls].log_prob(c_out)
                         for i_cls in range(len(cls_dists))],
                        dim=-1) for c_out in c_outs]
        data_pred_labels_per_class = [np.argmax(var_to_np(data_preds), axis=1)
                            for data_preds in data_preds_per_class]
        data_acc = np.mean(labels == np.concatenate(data_pred_labels_per_class))

        print("{:s} Accuracy: {:.1f}%".format(setname, acc * 100))
        fig = plt.figure(figsize=(5,5))
        ax = plt.gca()
        for i_class in range(len(c_outs)):
            #if i_class == 0:
            #    continue
            o = var_to_np(c_outs[i_class]).squeeze()
            incorrect_pred_mask = pred_labels_per_class[i_class] != i_class
            plt.scatter(o[:,0], o[:,1], s=20, alpha=0.75, label=["Right", "Rest"][i_class])
            assert len(incorrect_pred_mask) == len(o)
            plt.scatter(o[incorrect_pred_mask,0], o[incorrect_pred_mask,1], marker='x', color='black',
                       alpha=1, s=5)
            means, stds = class_dist.get_mean_std(i_class)
            means = var_to_np(means)[:2]
            stds = var_to_np(stds)[:2]
            for sigma in [0.5,1,2,3]:
                ellipse = Ellipse(means, stds[0]*sigma, stds[1]*sigma)
                ax.add_artist(ellipse)
                ellipse.set_edgecolor(seaborn.color_palette()[i_class])
                ellipse.set_facecolor("None")
        for i_class in range(len(c_outs)):
            o = var_to_np(c_outs[i_class]).squeeze()
            plt.scatter(np.mean(o[:,0]), np.mean(o[:,1]),
                       color=seaborn.color_palette()[i_class+2], s=80, marker="^",
                       label=["Right Mean", "Rest Mean"][i_class])

        plt.title("{:6s} Accuracy:        {:.1f}%\n"
                  "From data mean/std: {:.1f}%".format(setname, acc * 100, data_acc * 100))
        plt.legend(bbox_to_anchor=(1,1,0,0))
        display_close(fig)
    return

In [ ]:
from reversible2.plot import display_close

In [ ]:
from reversible2.timer import Timer
from plot import plot_outs

i_start_epoch_out = 200
n_epochs = 1001
for i_epoch in range(n_epochs):
    with Timer(name='EpochLoop', verbose=False) as loop_time:
        optim_model.zero_grad()
        optim_dist.zero_grad()
        for i_class in range(len(train_inputs)):
            class_ins = train_inputs[i_class]
            samples = class_dist.get_samples(i_class, len(train_inputs[i_class]) * 5)
            inverted = invert(feature_model, samples)
            outs = feature_model(class_ins)
            if i_epoch < i_start_epoch_out:
                ot_loss_out = th.zeros(1, device=class_ins.device)
            else:
                ot_loss_out = ot_euclidean_loss_for_samples(outs[:,:2].squeeze(), samples[:,:2].squeeze())
            ot_loss_in = ot_euclidean_loss_for_samples(class_ins.squeeze(), inverted.squeeze())

            other_class_ins = train_inputs[1-i_class]
            changed_to_other_class = class_dist.change_to_other_class(outs, i_class_from=i_class, i_class_to=1-i_class)
            other_inverted = invert(feature_model, changed_to_other_class)
            ot_transformed_in = ot_euclidean_loss_for_samples(other_class_ins.squeeze(), other_inverted.squeeze())
            if i_epoch < i_start_epoch_out:
                ot_transformed_out = th.zeros(1, device=class_ins.device)
            else:
                other_samples = class_dist.get_samples(1-i_class, len(train_inputs[i_class]) * 5)
                ot_transformed_out = ot_euclidean_loss_for_samples(changed_to_other_class[:,:2].squeeze(),
                                                                   other_samples[:,:2].squeeze(),)
            loss = ot_loss_in + ot_loss_out + ot_transformed_in + ot_transformed_out
            loss.backward()
        optim_model.step()
        optim_dist.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("Loss: {:.2E}".format(loss.item()))
        print("OT Loss In: {:.2E}".format(ot_loss_in.item()))
        print("OT Loss Out: {:.2E}".format(ot_loss_out.item()))
        print("Transformed OT Loss In: {:.2E}".format(ot_transformed_in.item()))
        print("Transformed OT Loss Out: {:.2E}".format(ot_transformed_out.item()))
        print("Loop Time: {:.0f} ms".format(loop_time.elapsed_secs * 1000))
        plot_outs(feature_model, train_inputs, test_inputs,
                 class_dist)
        fig = plt.figure(figsize=(8,2))
        plt.plot(var_to_np(th.cat((th.exp(class_dist.class_log_stds),
                                 th.exp(class_dist.non_class_log_stds)))),
                marker='o')
        display_close(fig)

In [ ]:
fig = plt.figure(figsize=(5,4))
for i_class in range(len(train_inputs)):
    class_ins = train_inputs[i_class]
    train_outs = feature_model(class_ins)
    plt.scatter(var_to_np(train_outs)[:,0], var_to_np(train_outs)[:,1],
               color=seaborn.color_palette()[i_class])
for i_class in range(len(train_inputs)):
    class_ins = test_inputs[i_class]
    test_outs = feature_model(class_ins)
    plt.scatter(var_to_np(test_outs)[:,0], var_to_np(test_outs)[:,1],
               color=seaborn.color_palette()[i_class+2])
    
display_close(fig)